In [1]:
from fastfusion import Specification
spec = Specification.from_yaml(
    "architecture/four_level.arch.yaml",
    "workloads/mha_full_new.yaml"
)
# spec.estimate_energy_area()
# spec.component_energy.to_yaml("ERT.yaml")
# spec.component_area.to_yaml("ART.yaml")
# spec.workload.mermaid_graph() # pip3 install mermaid-py
# spec.workload.einsums

2025-05-07 15:23:29 INFO        Loading yaml file architecture/four_level.arch.yaml
2025-05-07 15:23:29 INFO        Found top key variables in architecture/four_level.arch.yaml
2025-05-07 15:23:29 INFO        Found top key architecture in architecture/four_level.arch.yaml
2025-05-07 15:23:29 INFO        Found top key compound_components in architecture/four_level.arch.yaml
2025-05-07 15:23:29 INFO        Loading yaml file workloads/mha_full_new.yaml
2025-05-07 15:23:29 INFO        Found top key workload in workloads/mha_full_new.yaml
2025-05-07 15:23:29 WARNING     Loading configuration file from /root/.config/fastfusion/config.yaml
2025-05-07 15:23:29 INFO        Loading yaml file /root/.config/fastfusion/config.yaml
2025-05-07 15:23:29 INFO        Found top key version in /root/.config/fastfusion/config.yaml
2025-05-07 15:23:29 INFO        Found top key environment_variables in /root/.config/fastfusion/config.yaml
2025-05-07 15:23:29 INFO        Found top key expression_custom_functi

In [2]:
from fastfusion.frontend.renames import Renames
from fastfusion.frontend.workload import Workload
workload = Workload.from_yaml(
    "workloads/mha_full_new.yaml",
)
renames = Renames.from_yaml(
    "workloads/mha_full_new.renames.yaml",
)
for e in workload.einsums:
    symbol_table = workload.get_constraint_symbol_table(e.name, renames)
    print(f'Einsum {e.name}:')
    for k, v in symbol_table.items():
        print(f'\t{k}: {v}')

2025-05-07 15:23:29 INFO        Loading yaml file workloads/mha_full_new.yaml
2025-05-07 15:23:29 INFO        Found top key workload in workloads/mha_full_new.yaml
2025-05-07 15:23:29 WARNING     Trying to parse a single element dictionary as a Workload. 
2025-05-07 15:23:29 INFO        Loading yaml file workloads/mha_full_new.renames.yaml
2025-05-07 15:23:29 INFO        Found top key renames in workloads/mha_full_new.renames.yaml
2025-05-07 15:23:29 WARNING     Trying to parse a single element dictionary as a Renames. 


Einsum V:
	All: {Tensor(I), Tensor(V), Tensor(WV)}
	Inputs: {Tensor(I), Tensor(WV)}
	Outputs: {Tensor(V)}
	Intermediates: {Tensor(V)}
	Shared: {Tensor(I), Tensor(V)}
	WV: {Tensor(WV)}
	V: {Tensor(V)}
	I: {Tensor(I)}
	weight: {Tensor(WV)}
	output: {Tensor(V)}
	input: {Tensor(I)}
Einsum K:
	All: {Tensor(I), Tensor(K), Tensor(WK)}
	Inputs: {Tensor(I), Tensor(WK)}
	Outputs: {Tensor(K)}
	Intermediates: {Tensor(K)}
	Shared: {Tensor(I), Tensor(K)}
	WK: {Tensor(WK)}
	I: {Tensor(I)}
	K: {Tensor(K)}
	weight: {Tensor(WK)}
	output: {Tensor(K)}
	input: {Tensor(I)}
Einsum Q:
	All: {Tensor(I), Tensor(Q), Tensor(WQ)}
	Inputs: {Tensor(I), Tensor(WQ)}
	Outputs: {Tensor(Q)}
	Intermediates: {Tensor(Q)}
	Shared: {Tensor(I), Tensor(Q)}
	Q: {Tensor(Q)}
	WQ: {Tensor(WQ)}
	I: {Tensor(I)}
	weight: {Tensor(WQ)}
	output: {Tensor(Q)}
	input: {Tensor(I)}
Einsum QK:
	All: {Tensor(K), Tensor(Q), Tensor(QK)}
	Inputs: {Tensor(K), Tensor(Q)}
	Outputs: {Tensor(QK)}
	Intermediates: {Tensor(K), Tensor(Q), Tensor(QK)}
	Shar

In [ ]:
einsum = "QK"
symbol_table = workload.get_constraint_symbol_table(einsum, renames)
first_value = next(iter(symbol_table.values()))
arch_nodes = spec.get_flattened_architecture()

for node in arch_nodes:
    storage_constraints = node.constraints.storage._parse(symbol_table)
    print(f'{node.name}:')
    for k, v in storage_constraints.items():
        print(f'\t{k}: {v}')

    keep_choice = first_value.to_my_space(set(("Q", "K")))
    keep_choice.tensors = lambda: keep_choice # So users can do MainMemory().tensors(). Optional.
    symbol_table[node.name] = keep_choice

    temporal = node.constraints.temporal._parse(symbol_table)
    spatial_X = node.constraints.get_spatial_constraint(for_X=True)._parse(symbol_table)
    spatial_Y = node.constraints.get_spatial_constraint(for_Y=True)._parse(symbol_table)

    for k, v in temporal.items():
        print(f'\tTemporal {k}: {v}')
    for k, v in spatial_X.items():
        print(f'\tSpatial X {k}: {v}')
    for k, v in spatial_Y.items():
        print(f'\tSpatial Y {k}: {v}')

2025-05-07 15:23:29 WARNING     Loading configuration file from /root/.config/fastfusion/config.yaml
2025-05-07 15:23:30 INFO        Loading yaml file /root/.config/fastfusion/config.yaml
2025-05-07 15:23:30 INFO        Found top key version in /root/.config/fastfusion/config.yaml
2025-05-07 15:23:30 INFO        Found top key environment_variables in /root/.config/fastfusion/config.yaml
2025-05-07 15:23:30 INFO        Found top key expression_custom_functions in /root/.config/fastfusion/config.yaml
2025-05-07 15:23:30 INFO        Found top key component_plug_ins in /root/.config/fastfusion/config.yaml
2025-05-07 15:23:30 INFO        Specification processed in 0.00 seconds
2025-05-07 15:23:30 INFO        Calculated Specification[variables].Variables[version] as "0.5" = 0.5.
2025-05-07 15:23:30 INFO        Calculated Specification[architecture].Architecture[nodes].ArchNodes[0].Storage(MainMemory)[attributes].StorageAttributes[energy_scale] as "(450 / 64) / 8" = 0.87890625.
2025-05-07 15:

MainMemory:
	bypass: {}
	keep: {}
	coalesce: {Tensor(K), Tensor(Q), Tensor(QK)}
	Temporal loop_bounds: LoopBoundsList=[]
	Temporal loop_order: LoopOrder=[]
	Temporal reuse: All
	Temporal rmw_first_update: {}
	Spatial X loop_bounds: LoopBoundsList=[]
	Spatial X loop_order: LoopOrder=[]
	Spatial X reuse: All
	Spatial Y loop_bounds: LoopBoundsList=[]
	Spatial Y loop_order: LoopOrder=[]
	Spatial Y reuse: All
GlobalBuffer:
	bypass: {}
	keep: {Tensor(QK)}
	coalesce: {Tensor(K), Tensor(Q), Tensor(QK)}
	Temporal loop_bounds: LoopBoundsList=[]
	Temporal loop_order: LoopOrder=[]
	Temporal reuse: All
	Temporal rmw_first_update: {}
	Spatial X loop_bounds: LoopBoundsList=[]
	Spatial X loop_order: LoopOrder=[]
	Spatial X reuse: All
	Spatial Y loop_bounds: LoopBoundsList=[]
	Spatial Y loop_order: LoopOrder=[]
	Spatial Y reuse: All
LocalBuffer:
	bypass: {}
	keep: {Tensor(K), Tensor(Q), Tensor(QK)}
	coalesce: {Tensor(K), Tensor(Q), Tensor(QK)}
	Temporal loop_bounds: LoopBoundsList=[]
	Temporal loop_ord